# RUN TEST SIMULATION

## Control run

* Prepare run directory: wps, wrf

* Namelist setting

* Run geogrid, ungrib, metgrid, real and wrf


In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import os, sys, glob
import subprocess

### Preparing run directory


In [49]:
wpsdir              = "/Volumes/work/WRF_program/WRF_install/WPS/"
wrfdir              = "/Volumes/work/WRF_program/WRF_install/WRF_mpi/"
geogdir             = "/Volumes/work/WRF_program/WPS_GEOG"
renaldir            = "/Volumes/work/WRF_program/era5/"
namelist_wps_path   = "namelist/namelist.wps"
namelist_input_path = "namelist/namelist.input"

run_id              = 'tok_test_dom'
base_dir            = '/Users/doan/Google Drive/My Drive/share/2024/PGW_DS/'
run_dir             = base_dir + "/Run_WRF/"+run_id+"control"

In [3]:
ls

01_Create_GWI_files.ipynb             NUL
01_Create_GWI_files.py                Readme.md
01_Generate_Global_Warming_Increment/ Run_WRF.ipynb
02_Create_PGW_WRFinterp.ipynb         WRF_run/
Download_CMIP6/                       namelist/


In [4]:
st_date, en_date = "2023-01-01 00", "2023-01-01 18"

In [5]:
import shutil

for idir in ['geogrid', 'ungrib', 'metgrid']: 
    # Copy the directory tree, allowing existing destination directory
    shutil.copytree(wpsdir+'/'+idir, run_dir+'/'+idir, dirs_exist_ok=True)
    shutil.copy2(wpsdir+'/'+idir+'.exe', run_dir+'/'+idir+'.exe')    

shutil.copy2(wpsdir+'/link_grib.csh', run_dir)

'/Users/doan/Google Drive/My Drive/share/2024/PGW_DS//Run_WRF/control/link_grib.csh'

## modify name list


In [6]:
import re
def modify_namelist(namelist_path_in, namelist_path_out, aa):
    with open(namelist_path_in, 'r') as file:
        lines = file.readlines()

    for i, line in enumerate(lines):
        for k, v in aa.items():
            parameter_pattern = re.compile(rf'^\s*{k}\s*=\s*')
            if parameter_pattern.match(line):
                lines[i] = f' {k} = {v},\n'

    with open(namelist_path_out, 'w') as file:
        file.writelines(lines)

In [7]:
namelist_path_in = namelist_wps_path
namelist_path_out = run_dir+'/namelist.wps'
aa = {'geog_data_path': '"'+geogdir+'"' }

modify_namelist(namelist_path_in, namelist_path_out, aa)


### Run geogrid.exe

In [8]:
try:
    subprocess.run(['./geogrid.exe'], cwd=run_dir, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
except subprocess.CalledProcessError as e:
    print(f"Error executing geogrid.exe: {e}")

In [9]:
ls

01_Create_GWI_files.ipynb             NUL
01_Create_GWI_files.py                Readme.md
01_Generate_Global_Warming_Increment/ Run_WRF.ipynb
02_Create_PGW_WRFinterp.ipynb         WRF_run/
Download_CMIP6/                       namelist/


In [10]:
#os.system('cd '+run_dir+'; ./geogrid.exe  > NUL 2>&1; cd -')

In [11]:
### Run ungrib

In [12]:
ls

01_Create_GWI_files.ipynb             NUL
01_Create_GWI_files.py                Readme.md
01_Generate_Global_Warming_Increment/ Run_WRF.ipynb
02_Create_PGW_WRFinterp.ipynb         WRF_run/
Download_CMIP6/                       namelist/


In [13]:
source_file = os.path.join(run_dir, 'ungrib/Variable_Tables/Vtable.ECMWF')
target_link = os.path.join(run_dir, 'Vtable')
original_dir = os.getcwd()
# Create the symbolic link
try:
    # Change to the run directory (run_dir)
    os.chdir(run_dir)
    # Remove existing symbolic link or file if it exists
    if os.path.islink(target_link) or os.path.exists(target_link):
        os.remove(target_link)
    # Create a symbolic link
    os.symlink(source_file, target_link)
    print(f"Symbolic link created from {source_file} to {target_link}")
except OSError as e:
    print(f"Error creating symbolic link: {e}")
finally:
    # Move back to the original directory
    os.chdir(original_dir)
    print(f"Moved back to the original directory: {original_dir}")

Symbolic link created from /Users/doan/Google Drive/My Drive/share/2024/PGW_DS//Run_WRF/control/ungrib/Variable_Tables/Vtable.ECMWF to /Users/doan/Google Drive/My Drive/share/2024/PGW_DS//Run_WRF/control/Vtable
Moved back to the original directory: /Users/doan/Documents/GitHub/Pseudo-Global-Warming-Downscaling


In [29]:
#os.system('cd '+run_dir+';ln -sf ungrib/Variable_Tables/Vtable.ECMWF Vtable')

0

In [42]:
import subprocess

namelist_wps = run_dir + '/namelist.wps'
prefixes = ['ERA5A', 'ERA5S']

for i in range(2):

    prefix = prefixes[i]
    # Define the appropriate reanalysis file based on 'pressure' or 'surface' level
    reanal_files = renaldir + 'era5_ungrib_' + ['pressure', 'surface'][i] + '_levels_20230101.grib'
    # Run the link_grib.csh script using subprocess, passing the reanalysis file
    try:
        subprocess.run(['./link_grib.csh', reanal_files], cwd=run_dir, check=True)
        subprocess.run(['rm', '-f'] + glob.glob(os.path.join(run_dir, prefix+'*')), check=True)
                
    except subprocess.CalledProcessError as e:
        print(f"Error running link_grib.csh: {e}")

    # Modify the namelist
    modify_namelist(namelist_wps, namelist_wps, {'prefix': f'"{prefix}"'})
    
    # Run ungrib.exe and suppress output (equivalent to '> NUL 2>&1')
    try:
        subprocess.run(['./ungrib.exe'], cwd=run_dir, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error running ungrib.exe: {e}")

In [32]:
'''
namelist_wps = run_dir+'/namelist.wps'
prefixs = ['ERA5A', 'ERA5S']

for i in range(2):

    prefix = prefixs[i]
    reanal_files = renaldir + 'era5_ungrib_'+['pressure', 'surface'][i]+'_levels_20230101.grib'
    os.system('cd '+run_dir+';./link_grib.csh '+reanal_files)
    modify_namelist(namelist_wps, namelist_wps, { 'prefix'  :'"'+prefix+'"'  } )
    os.system('cd '+run_dir+'; ./ungrib.exe  > NUL 2>&1; cd -')
    #os.system('cd '+run_dir+'; ./ungrib.exe ') 
''' 

/Users/doan/Documents/GitHub/Pseudo-Global-Warming-Downscaling/03_WRF_run
/Users/doan/Documents/GitHub/Pseudo-Global-Warming-Downscaling/03_WRF_run


### Run metgrid

In [43]:
fg_name = '"'+'","'.join(prefixes)+'"'

modify_namelist(namelist_wps, namelist_wps, { 'fg_name'  : fg_name} )

In [44]:

try:
    # Run the command in the specified directory
    subprocess.run(['./metgrid.exe'], cwd=run_dir, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
except subprocess.CalledProcessError as e:
    print(f"Error executing metgrid.exe: {e}")

In [34]:
#os.system('cd '+run_dir+'; ./metgrid.exe  > NUL 2>&1; cd -')

/Users/doan/Documents/GitHub/Pseudo-Global-Warming-Downscaling/03_WRF_run


0

## Copy WRF run files

In [45]:
# List all files in the source directory (wrfdir + '/run/')
source_dir = os.path.join(wrfdir, 'run')
destination_dir = run_dir

# Copy all files from source_dir to destination_dir
for file_name in os.listdir(source_dir):
    full_file_name = os.path.join(source_dir, file_name)
    
    # Ensure it's a file before copying
    if os.path.isfile(full_file_name):
        shutil.copy(full_file_name, destination_dir)

cat: namelist.input: No such file or directory


In [48]:
namelist_input_path

'namelist.input'

In [50]:
import re
with open(namelist_input_path, 'r') as file: a1 = file.readlines()
with open(run_dir+'/namelist.input', 'w') as file: file.writelines(a1)

In [51]:
try:
    subprocess.run(['./real.exe'], cwd=run_dir, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
except subprocess.CalledProcessError as e:
    print(f"Error executing real.exe: {e}")

In [35]:
#os.system('cp '+wrfdir+'/run/* '+ run_dir+'/')

0

In [52]:
command = ['mpirun', '-np', '4', './wrf.exe']
try:
    subprocess.run(command, cwd=run_dir, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
except subprocess.CalledProcessError as e:
    print(f"Error executing wrf.exe: {e}")